<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/models/studio_3/DDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install optuna 

In [ ]:
import pickle
import pandas as pd
import optuna
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
path="Desktop\\DDI"

In [ ]:
path="/content/drive/Shareddrives/Deep Learning/datasets/DDI/"

PATH PER WINDOWDS



In [ ]:
dev = pd.read_csv(path+'\ddi2013-type\\dev.tsv', sep='\t')
test = pd.read_csv(path+'\ddi2013-type\\test.tsv', sep='\t')
train = pd.read_csv(path+'\ddi2013-type\\train.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"\ddi2013-type\\DDI_sinonimi_test.csv")
data_embedding = pd.read_csv(path+"\ddi2013-type\\DDI_embedding_test.csv")

data_embedding_train = pd.read_csv(path+"\ddi2013-type\\DDI_embedding.csv")
data_sinonimi_train = pd.read_csv(path+"\ddi2013-type\\DDI_sinonimi.csv")

print(test.shape)
print(data_sinonimi.shape)
print(test)

In [ ]:
dev = pd.read_csv(path+'ddi2013-type/dev.tsv', sep='\t')
test = pd.read_csv(path+'ddi2013-type/test.tsv', sep='\t')
train = pd.read_csv(path+'ddi2013-type/train.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"ddi2013-type/DDI_sinonimi_test.csv")
data_embedding = pd.read_csv(path+"ddi2013-type/DDI_embedding_test.csv")

data_embedding_train = pd.read_csv(path+"ddi2013-type/DDI_embedding.csv")

print(test.shape)
print(data_sinonimi.shape)
print(test)

In [ ]:
test_study_1 = pd.concat([data_sinonimi, data_embedding], ignore_index=True)
train_study_1 = pd.concat([data_sinonimi_train, data_embedding_train], ignore_index=True)
print (test_study_1.shape)
print (train_study_1.shape)

(11522, 3)
(37558, 3)


In [ ]:
print(train.shape)
print(data_sinonimi.shape)
print(data_embedding.shape)

In [ ]:
print(train.label.value_counts())
print(data_sinonimi.label.value_counts())
print(data_embedding.label.value_counts())

In [ ]:
with open(path+"\word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open(path+"\embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

Set up per allenamento Modello

In [ ]:
categories = [['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int']]

my_text_to_word_sequence = lambda sen: keras.preprocessing.text.text_to_word_sequence(sen,
                                                                                      filters='!"#&()*+,-./:;<=>?[\\]^_`\'{|}~\t\n',
                                                                                      lower=True)

TRAINSET

In [ ]:
five_hot_train = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  train_study_1.label.to_numpy().reshape(-1, 1)
)

In [ ]:
sentences_train = [my_text_to_word_sequence(sentence) for sentence in train_study_1['sentence']]

TESTSET

In [ ]:
five_hot_test = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test_study_1.label.to_numpy().reshape(-1, 1)
)

In [ ]:
sentences_test = [my_text_to_word_sequence(sentence) for sentence in test_study_1['sentence']]

ORIGINALE

In [ ]:
five_hot_orig = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test.label.to_numpy().reshape(-1, 1)
)

In [ ]:
sentences_orig = [my_text_to_word_sequence(sentence) for sentence in test['sentence']]

SINONIMI

In [ ]:
five_hot_sin = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_sinonimi.label.to_numpy().reshape(-1, 1)
)

In [ ]:
sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

EMBEDDING

In [ ]:
five_hot_emb = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_embedding.label.to_numpy().reshape(-1, 1)
)

In [ ]:
sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

Estrai la massima dimensione dell'input in base ai vari dataset considerati

In [ ]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences_train):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_test):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_orig):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
  
print(f'Il massimo è {max}')

Crao i vari embedding per tutti i dataset, quest'operazione e pesante 

In [ ]:
embedded_trainset = np.zeros(shape=(len(sentences_train), max, 300))
for i, sentence in enumerate(sentences_train):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [ ]:
embedded_origin = np.zeros(shape=(len(sentences_orig), max, 300))
for i, sentence in enumerate(sentences_orig):
  for j, word in enumerate(sentence):
    try:
      embedded_origin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [ ]:
embedded_testset = np.zeros(shape=(len(sentences_test), max, 300))
for i, sentence in enumerate(sentences_test):
  for j, word in enumerate(sentence):
    try:
      embedded_testset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [ ]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
  for j, word in enumerate(sentence):
    try:
      embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [ ]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
  for j, word in enumerate(sentence):
    try:
      embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

Carica optuna results e inizializza il modello, oppure salva il modello oppure carica solo i pesi del modello 

In [ ]:
best_params = optuna.load_study(study_name="DDI", storage="sqlite:///"+path+"\ddi2013-type\\optuna_ddi_studio_0.db").best_params

In [ ]:
print(f'{best_params}')

{'batch_size': 89, 'dropout': 0.63, 'units': 81}


In [ ]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params['units'],
                                                             recurrent_dropout=best_params['dropout'],
                                                             activation='tanh')))

model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
#train the model
result = model.fit(embedded_trainset,
                   five_hot_train,
                   epochs=100,
                   batch_size=best_params['batch_size'],
                   callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                                                            patience=10)])

Epoch 1/100
422/422 [==============================] - 96s 227ms/step - loss: 0.5879 - accuracy: 0.8417
Epoch 2/100
422/422 [==============================] - 98s 232ms/step - loss: 0.4845 - accuracy: 0.8451
Epoch 3/100
422/422 [==============================] - 96s 227ms/step - loss: 0.4403 - accuracy: 0.8472
Epoch 4/100
422/422 [==============================] - 90s 213ms/step - loss: 0.4146 - accuracy: 0.8492
Epoch 5/100
422/422 [==============================] - 90s 214ms/step - loss: 0.3934 - accuracy: 0.8528
Epoch 6/100
422/422 [==============================] - 92s 219ms/step - loss: 0.3795 - accuracy: 0.8535
Epoch 7/100
422/422 [==============================] - 92s 218ms/step - loss: 0.3655 - accuracy: 0.8567
Epoch 8/100
422/422 [==============================] - 91s 215ms/step - loss: 0.3533 - accuracy: 0.8601
Epoch 9/100
422/422 [==============================] - 91s 215ms/step - loss: 0.3441 - accuracy: 0.8624
Epoch 10/100
422/422 [==============================] - 91s 215m

In [ ]:
#save the model
model.save_weights(path+'\ddi2013-type\\DDI_w_studio_3.h5')

In [ ]:
#load only the w of the model, the model must be already executed
model.load_weights(path+'\ddi2013-type\\DDI_w_studio_3.h5')

EVALUATE ALL DATASET

In [ ]:
result_base=model.evaluate(embedded_trainset, five_hot_train, batch_size=best_params['batch_size'],)
print(f'DATASET ORIGINARIO{result_base}')

422/422 [==============================] - 28s 65ms/step - loss: 0.0363 - accuracy: 0.9896
DATASET ORIGINARIO[0.036323703825473785, 0.9895894527435303]


In [ ]:
result_base=model.evaluate(embedded_testset, five_hot_test, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

130/130 [==============================] - 8s 63ms/step - loss: 0.9850 - accuracy: 0.8177
DATASET TEST[0.9849727749824524, 0.8177399635314941]


In [ ]:
result_base=model.evaluate(embedded_origin, five_hot_orig, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

65/65 [==============================] - 4s 63ms/step - loss: 0.8900 - accuracy: 0.7934
DATASET TEST[0.8900015354156494, 0.7934386134147644]


In [ ]:
result_base=model.evaluate(embedded_sin, five_hot_sin, batch_size=best_params['batch_size'],)
print(f'DATASET SINONIMI{result_base}')

65/65 [==============================] - 4s 62ms/step - loss: 1.0264 - accuracy: 0.8165
DATASET SINONIMI[1.0263546705245972, 0.8165249228477478]


In [ ]:
result_base=model.evaluate(embedded_emb, five_hot_emb, batch_size=best_params['batch_size'],)
print(f'DATASET EMBEDDING{result_base}')

65/65 [==============================] - 4s 66ms/step - loss: 0.9436 - accuracy: 0.8190
DATASET EMBEDDING[0.9435904026031494, 0.8189550638198853]


tali test sono stati effettuati su windows, eventuali path necessitano di essere adattati, se effettuati su sistema operativo unix like.

